In [1]:
!gpustat

auriga                   Sat Dec 10 18:57:17 2022  510.47.03
[0] Tesla V100-SXM2-32GB | 36'C,   0 % |  1825 / 32768 MB | a.lokis(1561M)
[1] Tesla V100-SXM2-32GB | 46'C,   3 % |  8806 / 32768 MB | e.trofimenko(4283M) e.trofimenko(4263M)
[2] Tesla V100-SXM2-32GB | 35'C,   0 % |  2547 / 32768 MB | a.lokis(2283M)
[3] Tesla V100-SXM2-32GB | 31'C,   0 % |   260 / 32768 MB |


In [2]:
import numpy as np
import pandas as pd
import os
import json
import shutil
from jiwer import wer, cer

import glob
import subprocess
import tarfile
import wget
import copy
from omegaconf import OmegaConf, open_dict

import wandb
from tqdm.auto import tqdm

from collections import defaultdict

import re
import unicodedata

import nemo
import nemo.collections.asr as nemo_asr
import nemo.collections.nlp as nemo_nlp
from nemo.collections.asr.metrics.wer import word_error_rate
from nemo.utils import logging, exp_manager

import torch
import pytorch_lightning as ptl

import time
import gensim
import compress_fasttext
from typing import Dict, List, Literal
from abc import ABC, abstractmethod
from Levenshtein import distance

[NeMo W 2022-12-11 13:43:20 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2022-12-11 13:43:21 nemo_logging:349] /home/a.lokis/miniconda3/envs/asr_env/lib/python3.9/site-packages/torch/jit/annotations.py:299: UserWarning: TorchScript will treat type annotations of Tensor dtype-specific subtypes as if they are normal Tensors. dtype constraints are not enforced in compilation either.
      warnings.warn("TorchScript will treat type annotations of Tensor "
    
[NeMo W 2022-12-11 13:43:21 experimental:27] Module <class 'nemo.collections.nlp.data.language_modeling.megatron.megatron_batch_samplers.MegatronPretrainingRandomBatchSampler'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2022-12-11 13:43:22 experimental:27] Module <class 'nemo.collections.nlp.models.text_normalization_as_tagging.thutmose_tagger.ThutmoseTaggerModel'> is experimental, not ready for production and is not fully su

In [3]:
def read_manifest(path):
    manifest = []
    with open(path, 'r') as f:
        for line in f:
            line = line.replace("\n", "")
            data = json.loads(line)
            manifest.append(data)
    return manifest

def softmax(logits):
    e = np.exp(logits - np.max(logits))
    return e / e.sum(axis=-1).reshape([logits.shape[0], 1])

def test_asr_model(model, batch_size:int, manifests: List, message: str, 
                   probs: bool = False, cap_punkt_model = None):
    
    reg = re.compile('[^а-я ]')
    
    test_text = []
    test_path = []
    for path in manifests:
        mn = read_manifest(path)
        
        for sample in mn:
            
            clear_text = sample['text']
            clear_text = reg.sub('', sample['text'].lower().strip())
            ' '.join(clear_text.split())
            
            if clear_text != '':
                test_text.append(clear_text) 
                test_path.append(sample['audio_filepath'])
    
    if probs:
        logits = model.transcribe(paths2audio_files=test_path, 
                                            batch_size=batch_size, num_workers=16, logprobs=True)
        probs = []
        for sample in logits:
            probs.append(softmax(sample))
        probs = np.array(probs, 'dtype=object')
        described_text = None
        
    else:
        described_text = model.transcribe(paths2audio_files=test_path, 
                                            batch_size=batch_size, num_workers=32)
        probs = None

    try:
        WER = wer(test_text, described_text)
        CER = cer(test_text, described_text)

        print(f'{message}:')
        print('WER:', WER)
        print('CER:', CER, '\n')
        
        return test_text, described_text, probs
    except:
        print('Cannot calculate WER and CER')
        return test_text, described_text, probs

### save last ckpt in .nemo model 

ASR model

In [32]:
folder_path = 'experiments/ASR-Golos-Model-ru/2022-11-22_13-58-47/checkpoints/'
last_ckpt = os.listdir(folder_path)[-1]
print(last_ckpt)

ASR-Golos-Model-ru--val_wer=0.1062-epoch=50-last.ckpt


In [33]:
model = nemo.collections.asr.models.EncDecCTCModel.load_from_checkpoint(folder_path+last_ckpt)
model.save_to('ASR_models/golos_ft_withSpecAug_50epochs.nemo')

[NeMo W 2022-11-28 11:29:43 modelPT:142] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: manifests/ru/commonvoice_train_manifest_lower.json,manifests/ru/commonvoice_dev_manifest_lower.json,manifests/ru/commonvoice4_train_manifest_lower.json,manifests/ru/commonvoice4_dev_manifest_lower.json,manifests/ru/commonvoice5_train_manifest_lower.json,manifests/ru/commonvoice5_dev_manifest_lower.json,manifests/ru/commonvoice9_train_manifest_lower.json,manifests/ru/commonvoice9_dev_manifest_lower.json,manifests/ru/commonvoice10_train_manifest_lower.json,manifests/ru/commonvoice10_dev_manifest_lower.json,manifests/ru/commonvoice11_train_manifest_lower.json,manifests/ru/commonvoice11_dev_manifest_lower.json,ruls/train/train_manifest_lower.json,ruls/dev/dev_manifest_lower.json
    sample_rate: 16000
    labels:
    - ' '
    - а
    - б
  

[NeMo I 2022-11-28 11:29:43 features:225] PADDING: 16


Cap Punkt model

In [ ]:
folder_path = 'experiments/ASR-Golos-Model-ru/2022-11-22_13-58-47/checkpoints/'
last_ckpt = os.listdir(folder_path)[-1]
print(last_ckpt)

## inference

In [4]:
mn_list = [f'manifests/ru/commonvoice_test_manifest.json', 
           f'manifests/ru/commonvoice4_test_manifest.json',
           f'manifests/ru/commonvoice5_test_manifest.json',
           f'manifests/ru/commonvoice9_test_manifest.json',
           f'manifests/ru/commonvoice10_test_manifest.json',
           f'manifests/ru/commonvoice11_test_manifest.json',
#            f'golos/test/crowd/manifest.json',                  
#            f'golos/test/farfield/manifest.json',                 
           f'ruls/test/test_manifest.json']

In [5]:
model = nemo.collections.asr.models.EncDecCTCModel.restore_from('ASR_models/golos_ft_withSpecAug_50epochs.nemo')

[NeMo W 2022-11-30 13:08:57 modelPT:142] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: manifests/ru/commonvoice_train_manifest_lower.json,manifests/ru/commonvoice_dev_manifest_lower.json,manifests/ru/commonvoice4_train_manifest_lower.json,manifests/ru/commonvoice4_dev_manifest_lower.json,manifests/ru/commonvoice5_train_manifest_lower.json,manifests/ru/commonvoice5_dev_manifest_lower.json,manifests/ru/commonvoice9_train_manifest_lower.json,manifests/ru/commonvoice9_dev_manifest_lower.json,manifests/ru/commonvoice10_train_manifest_lower.json,manifests/ru/commonvoice10_dev_manifest_lower.json,manifests/ru/commonvoice11_train_manifest_lower.json,manifests/ru/commonvoice11_dev_manifest_lower.json,ruls/train/train_manifest_lower.json,ruls/dev/dev_manifest_lower.json
    sample_rate: 16000
    labels:
    - ' '
    - а
    - б
  

[NeMo I 2022-11-30 13:08:57 features:225] PADDING: 16
[NeMo I 2022-11-30 13:09:03 save_restore_connector:243] Model EncDecCTCModel was successfully restored from /home/projects/asr/ASR_models/golos_ft_withSpecAug_50epochs.nemo.


In [10]:
test_text, described_text, _ = test_asr_model(model = model, batch_size = 4,
                            manifests = mn_list, 
                            message = 'golos fine-tuned 50 epochs with SpecAugment')

Transcribing:   0%|          | 0/12885 [00:03<?, ?it/s]

golos fine-tuned 50 epochs with SpecAugment:
WER: 0.106251618384419
CER: 0.02671219908519743 



In [18]:
%%time
test_text, described_text, probs = test_asr_model(model = model, batch_size = 4,
                            manifests = mn_list, 
                            message = '', probs=True)

Transcribing:   0%|          | 0/12885 [00:04<?, ?it/s]

[NeMo W 2022-11-30 17:42:52 nemo_logging:349] /tmp/ipykernel_22694/2292581883.py:40: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
      probs = np.array(probs)
    


Cannot calculate WER and CER


In [25]:
np.save('golos_ft50e_probs.npy', probs)

In [58]:
for pair in zip(test_text[:100], described_text[:100]):
    print('real: ', pair[0])
    print('pred: ', pair[1])

real:  если не будет возражений я буду считать что ассамблея согласна с этим предложением
pred:  если не будет возражений я буду считать что ассамблея согласна с этим предложением
real:  палестинцам необходимо сначала установить мир с израилем а затем добиваться признания государственности
pred:  пороскинцам необходимо сначала установить мир фиврелем а затем дубираться признания государственности
real:  у меня было такое чувство что чтото такое очень важное я прибавляю
pred:  у меня было такое чувство что чтото такое очень важное я прибавляю
real:  только вряд ли поймет
pred:  только вряд ли поймет
real:  вронский слушая одним ухом переводил бинокль с бенуара на бельэтаж и оглядывал ложи
pred:  владкий слушая один однимуком приводилбы рокот спинва на прилетаж и оглядывал лозу
real:  к сожалению ситуация продолжает ухудшаться
pred:  к сожалению ситуация продолжает ухушаться
real:  вс жалование уходило на домашние расходы и на уплату мелких непереводившихся долгов
pred:  все жалованье ух

In [66]:
df = pd.DataFrame({'real': test_text, 'pred': described_text})
df.to_csv('inference_dataframes/golos50e_wer_11.tsv', sep='\t', index=False)
pd.read_csv('inference_dataframes/golos50e_wer_11.tsv', sep='\t').head()

,real,pred
0,если не будет возражений я буду считать что ас...,если не будет возражений я буду считать что ас...
1,палестинцам необходимо сначала установить мир ...,пороскинцам необходимо сначала установить мир ...
2,у меня было такое чувство что чтото такое очен...,у меня было такое чувство что чтото такое очен...
3,только вряд ли поймет,только вряд ли поймет
4,вронский слушая одним ухом переводил бинокль с...,владкий слушая один однимуком приводилбы рокот...


### Test CapPunkt model

In [17]:
path = 'NLP_models/LexCapPunkt20epochs.nemo' # NLP_models/basis_CapPunkt_model.nemo, NLP_models/LexCapPunkt20epochs.nemo
pretrained_model = nemo_nlp.models.PunctuationCapitalizationLexicalAudioModel.restore_from(path)

[NeMo I 2022-12-11 13:53:17 tokenizer_utils:130] Getting HuggingFace AutoTokenizer with pretrained_model_name: bert-base-uncased, vocab_file: /tmp/tmpi_t2wgov/35346f4f44504a64b606693320ef3afc_vocab.txt, merges_files: None, special_tokens_dict: {}, and use_fast: False


Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.
[NeMo W 2022-12-11 13:53:20 modelPT:222] You tried to register an artifact under config key=tokenizer.vocab_file but an artifact for it has already been registered.
[NeMo W 2022-12-11 13:53:20 modelPT:142] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    use_tarred_dataset: false
    ds_item: data_for_CapPunkt
    text_file: text_train.txt
    labels_file: labels_train.txt
    audio_file: audio_train.txt
    use_audio: true
    use_bucketing: true
    preload_audios: true
    tokens_in_batch: 1024
    max_seq_length: 512
    sample_rate: 16000
    num_workers: 32
    n_jobs: 0
    tar_metadata_file: null
    tar_shuffle_n: 1
    
[NeMo W 2022-12-11 13:53:20 modelPT:149] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_

[NeMo I 2022-12-11 13:53:24 cloud:56] Found existing object /home/a.lokis/.cache/torch/NeMo/NeMo_1.14.0/stt_ru_quartznet15x5/92506570b7206ea395e295b3fbbf07e3/stt_ru_quartznet15x5.nemo.
[NeMo I 2022-12-11 13:53:24 cloud:62] Re-using file from: /home/a.lokis/.cache/torch/NeMo/NeMo_1.14.0/stt_ru_quartznet15x5/92506570b7206ea395e295b3fbbf07e3/stt_ru_quartznet15x5.nemo
[NeMo I 2022-12-11 13:53:24 common:912] Instantiating model from pre-trained checkpoint
[NeMo I 2022-12-11 13:53:25 cloud:56] Found existing object /home/a.lokis/.cache/torch/NeMo/NeMo_1.14.0/stt_ru_quartznet15x5/92506570b7206ea395e295b3fbbf07e3/stt_ru_quartznet15x5.nemo.
[NeMo I 2022-12-11 13:53:25 cloud:62] Re-using file from: /home/a.lokis/.cache/torch/NeMo/NeMo_1.14.0/stt_ru_quartznet15x5/92506570b7206ea395e295b3fbbf07e3/stt_ru_quartznet15x5.nemo
[NeMo I 2022-12-11 13:53:25 common:912] Instantiating model from pre-trained checkpoint


[NeMo W 2022-12-11 13:53:25 modelPT:142] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /raid/noneval.json
    sample_rate: 16000
    labels:
    - ' '
    - а
    - б
    - в
    - г
    - д
    - е
    - ё
    - ж
    - з
    - и
    - й
    - к
    - л
    - м
    - н
    - о
    - п
    - р
    - с
    - т
    - у
    - ф
    - х
    - ц
    - ч
    - ш
    - щ
    - ъ
    - ы
    - ь
    - э
    - ю
    - я
    batch_size: 16
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    num_workers: 8
    pin_memory: true
    
[NeMo W 2022-12-11 13:53:25 modelPT:149] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation da

[NeMo I 2022-12-11 13:53:25 features:267] PADDING: 16
[NeMo I 2022-12-11 13:53:26 save_restore_connector:243] Model EncDecCTCModel was successfully restored from /home/a.lokis/.cache/torch/NeMo/NeMo_1.14.0/stt_ru_quartznet15x5/92506570b7206ea395e295b3fbbf07e3/stt_ru_quartznet15x5.nemo.
[NeMo I 2022-12-11 13:53:28 save_restore_connector:243] Model PunctuationCapitalizationLexicalAudioModel was successfully restored from /home/projects/asr/NLP_models/LexCapPunkt20epochs.nemo.


In [18]:
audio = ['datasets/ru/test/wav/common_voice11_ru_26587318.wav', 
         'datasets/ru/test/wav/common_voice11_ru_19468869.wav', 
         'datasets/ru/test/wav/common_voice11_ru_31356050.wav']
text = ['к сожалению эти предложения не нашли отражения в тексте',
        'если не будет возражений я буду считать что ассамблея согласна с этим предложением', 
        'как дела']

pretrained_model.add_punctuation_capitalization(text, 
                                                audio_queries = audio)

[NeMo I 2022-12-11 13:53:28 punctuation_capitalization_lexical_audio_model:328] Using batch size 3 for inference
[NeMo I 2022-12-11 13:53:28 punctuation_capitalization_infer_dataset:123] Max length: 64
[NeMo I 2022-12-11 13:53:28 data_preprocessing:404] Some stats of the lengths of the sequences:
[NeMo I 2022-12-11 13:53:28 data_preprocessing:406] Min: 7 |                  Max: 68 |                  Mean: 40.0 |                  Median: 45.0
[NeMo I 2022-12-11 13:53:28 data_preprocessing:412] 75 percentile: 56.50
[NeMo I 2022-12-11 13:53:28 data_preprocessing:413] 99 percentile: 67.54


100%|██████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 11.48batch/s]


['К сожалению, эти предложения не нашли отражения в тексте.',
 'Если не будет возражений, я буду считать, что Ассамблея согласна с этим предложением.',
 'Как дела?']